In [376]:
import pandas as pd
import numpy as np
import re
from rapidfuzz import fuzz, process

In [378]:
#MODULE VARIOS PRUDCTOS EN UNA FILA

df = pd.read_csv("prueba2.csv")

# Iteramos por todas las columnas del DataFrame
for column in df.columns:
    # Convertimos a cadenas y dividimos en listas solo si son cadenas
    df[column] = df[column].apply(
        lambda x: x.split('\n') if isinstance(x, str) else ([x] if pd.notna(x) else [np.nan])
    )

# Crear una lista vacía para almacenar las filas explotadas
df_exploded_list = []

# Para cada fila en el DataFrame original, aplicamos explode en todas las columnas
for _, row in df.iterrows():
    # Crear una lista de las listas de cada columna de la fila
    row_lists = [row[col] if isinstance(row[col], list) else [row[col]] for col in df.columns]

    # Aseguramos que todas las listas tengan la misma longitud
    max_len = max(len(lst) for lst in row_lists)
    row_lists = [lst + [None] * (max_len - len(lst)) for lst in row_lists]

    # Convertir la lista de listas a un DataFrame temporal
    exploded_row = pd.DataFrame(row_lists).T

    # Asignar los nombres de las columnas originales
    exploded_row.columns = df.columns

    # Agregar la fila explotada a la lista
    df_exploded_list.append(exploded_row)
# Concatenamos todas las filas explotadas en un único DataFrame
df_exploded_1 = pd.concat(df_exploded_list, ignore_index=True)


In [379]:
df_exploded_1

,Unnamed: 0,DESCRIPCIÓN,PRESENTACION,CODIGO DE BARRAS,PRECIO CON IVA,PRECIO SIN IVA,PUBLICO\nSUGERIDO
0,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",8054083017068,"$ 740,145.59","$ 611,690.58","$ 1,073,211.02"
1,20005289,HUMIRA AC x 2 (40mg) HUMIRA AC x 1 (80 mg),"2 LAPICERAS PRELL. X 0,4 ml",8054083014050,"$ 1,480,292.66","$ 1,223,382.36","$ 2,146,424.35"
2,20021980,None,"1 LAPICERA PRELL. X 0,8 ml",8054083017648,"$ 1,553,789.84","$ 1,284,123.82","$ 2,252,995.41"
3,20062091,RINVOQ 15MG RINVOQ 30MG,15 MG x 30 COMP REC DE LIB. PROLONGADA,8054083018706,"$ 1,799,606.71","$ 1,487,278.28","$ 2,609,429.58"
4,20069005,RINVOQ 45MG,30 MG x 30 COMP REC DE LIB. PROLONGADA,8054083021386,"$ 3,184,521.02","$ 2,631,835.62","$ 4,617,555.55"
5,20078725,None,45 MG x 28 COMP REC DE LIB. PROLONGADA,8054083023922,"$ 4,494,517.30","$ 3,714,477.10","$ 6,517,049.97"
6,20069017,SKYRIZI SKYRIZI,1 LAPICERA PRELLENADA por 1 ML (150 mg),8054083021935,"$ 4,344,159.74","$ 3,590,214.67","$ 6,299,031.64"
7,20079636,SKYRIZI,360mg /2.4ml Cart. prellenado x 1 600mg /10ml ...,8054083023311,"$ 4,561,367.72","$ 3,769,725.40","$ 6,613,983.22"
8,20079635,None,None,8054083023328,"$ 4,561,367.72","$ 3,769,725.40","$ 6,613,983.22"
9,20058578,KALETRA SOLUCIÓN,160 ML,8054083006406,"$ 148,177.93","$ 122,461.09","$ 214,857.98"


In [380]:
#MODULE UN PRUDCTO POR FILA

df = pd.read_csv("prueba.csv")

# Paso 1: Limpiar los saltos de línea \r\n y \n en la columna de descripción solo si existe
df['DESCRIPCIÓN'] = df['DESCRIPCIÓN'].apply(lambda x: x.replace('\r\n', ' ').replace('\n', ' ') if isinstance(x, str) else x)

# Paso 2: Detectar si en la columna 'DESCRIPCIÓN' hay más de un producto (detectando saltos de línea)
# Si hay saltos de línea, dividimos la celda en varias filas. Si no, dejamos la fila tal cual.
df['DESCRIPCIÓN'] = df['DESCRIPCIÓN'].apply(lambda x: x.split('\n') if isinstance(x, str) and '\n' in x else [x])

# Paso 3: Explosión de las filas
df_exploded_list = []

for _, row in df.iterrows():
    # Crear una lista de listas para cada columna
    row_lists = [row[col] if isinstance(row[col], list) else [row[col]] for col in df.columns]

    # Asegurarnos de que todas las listas tengan la misma longitud
    max_len = max(len(lst) for lst in row_lists)
    row_lists = [lst + [None] * (max_len - len(lst)) for lst in row_lists]

    # Convertir la lista de listas a un DataFrame temporal
    exploded_row = pd.DataFrame(row_lists).T

    # Asignar los nombres de las columnas originales
    exploded_row.columns = df.columns

    # Agregar la fila explotada a la lista
    df_exploded_list.append(exploded_row)

# Concatenamos todas las filas explotadas en un único DataFrame
df_exploded_2 = pd.concat(df_exploded_list, ignore_index=True)



In [381]:
df_exploded_2

,EAN,DESCRIPCIÓN,PVP,DROGUERIA,DTO.,CANT.X BULTO
0,7791939000089,ASPIRINA 100 Estuche: 10 blisters x 10 comp. x...,"$4.904,38","$2.305,06",53%,50
1,7791939001116,CARDIO VENT3 30 comp masticables x 100 mg,"$1,31",$393,70%,50
2,7791939000096,CALCIO BASE Estuche: 3 blisters x 20 comp. mas...,"$12.995,10","$6.107,70",53%,90
3,7791939000133,CARBON VEGETAL ACTIVADO Estuche: 10 blisters x...,"$14.071,69","$6.613,69",53%,50
4,7791939001383,CLONAZEPAM SL ...,"$ 6.208,60","$ 1.738,41",72%,40
5,7791939001390,CLONAZEPAM ...,"$ 11.824,64","$ 3.310,90",72%,40
6,7791939001413,CLONAZEPAM ...,"$ 11.326,00","$ 3.171,28",72%,40
7,7791939001437,CLONAZEPAM ...,"$ 12.593,93","$ 3.526,30",72%,40
8,7791939000416,IBUPROFENO 400 Estuche: 2 blisters x 10 comp. ...,"$3.100,34","$868,10",72%,50
9,7791939000522,IBUPROFENO 600 Estuche: 2 blisters x 10 comp. ...,"$6.567,98","$1.839,03",72%,40


In [382]:
df_exploded

,EAN,DESCRIPCIÓN,PVP,DROGUERIA,DTO.,CANT.X BULTO
0,7791939000089,ASPIRINA 100 Estuche: 10 blisters x 10 comp. x...,"$4.904,38","$2.305,06",53%,50
1,7791939001116,CARDIO VENT3 30 comp masticables x 100 mg,"$1,31",$393,70%,50
2,7791939000096,CALCIO BASE Estuche: 3 blisters x 20 comp. mas...,"$12.995,10","$6.107,70",53%,90
3,7791939000133,CARBON VEGETAL ACTIVADO Estuche: 10 blisters x...,"$14.071,69","$6.613,69",53%,50
4,7791939001383,CLONAZEPAM SL ...,"$ 6.208,60","$ 1.738,41",72%,40
5,7791939001390,CLONAZEPAM ...,"$ 11.824,64","$ 3.310,90",72%,40
6,7791939001413,CLONAZEPAM ...,"$ 11.326,00","$ 3.171,28",72%,40
7,7791939001437,CLONAZEPAM ...,"$ 12.593,93","$ 3.526,30",72%,40
8,7791939000416,IBUPROFENO 400 Estuche: 2 blisters x 10 comp. ...,"$3.100,34","$868,10",72%,50
9,7791939000522,IBUPROFENO 600 Estuche: 2 blisters x 10 comp. ...,"$6.567,98","$1.839,03",72%,40
